  # Creat Kfolds

In [0]:
!pip install iterative-stratification

In [0]:
import pandas as pd 
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [0]:
df = pd.read_csv('/content/Kaggle/train.csv')
df["kfold"] = -1
df = df.sample(frac=1).reset_index(drop=True)
kf = MultilabelStratifiedKFold(n_splits=5)
X=df.image_id.values 
y = df[['grapheme_root',	'vowel_diacritic'	,'consonant_diacritic']].values
for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
  print(len(train_idx), len(val_idx))
  df.loc[val_idx, 'kfold'] = fold
df.to_csv("/content/drive/My Drive/Kaggle/train_folds.csv", index=False)

160672 40168
160672 40168
160672 40168
160672 40168
160672 40168


# creat PKL 

In [0]:
import joblib 
import glob 
from tqdm import tqdm 

In [0]:
## get image pickle 
files = glob.glob('/content/Kaggle/train_*.parquet')
for f in files : 
  df = pd.read_parquet(f) 
  image_ids = df.image_id.values 
  df = df.drop('image_id',1) 
  image_array =df.values 
  for j , img_id in tqdm(enumerate(image_ids) , total = len(image_ids) ) : 
    joblib.dump(image_array[j, :],f"/content/Kaggle/images/{img_id}.pkl")

100%|██████████| 50210/50210 [00:58<00:00, 863.50it/s]


# Dataset Class

In [0]:
!pip install albumentations

     |████████████████████████████████| 634kB 2.8MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=1669167a4b4f975f26e54245a996ded3ea0f569a07982fe80dc6e8f91d318ded
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision


     |████████████████████████████████| 592.3MB 1.2MB/s 
ERROR: torchvision 0.5.0 has requirement torch==1.4.0, but you'll have torch 0.3.0.post4 which is incompatible.
ERROR: fastai 1.0.60 has requirement torch>=1.0.0, but you'll have torch 0.3.0.post4 which is incompatible.
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
     |████████████████████████████████| 753.4MB 23kB/s 
  Found existing installation: torch 0.3.0.post4
    Uninstalling torch-0.3.0.post4:
      Successfully uninstalled torch-0.3.0.post4


In [0]:
import joblib 
import pandas as pd 
from PIL import Image
import albumentations 
import numpy as np 
import torch 
class BengaliDatasetTrain : 
  def __init__(self,folds,img_height=137,img_width=236,mean=(0.485,0.456,0.405),std=(0.229,0.224,0.225)) : 
    df = pd.read_csv("/content/drive/My Drive/Kaggle/train_folds.csv")
    df =df[['image_id','grapheme_root',	'vowel_diacritic'	,'consonant_diacritic',"kfold"]]
    df = df[df.kfold.isin(folds)].reset_index(drop=True) 
    self.image_ids = df.image_id.values 
    self.grapheme_root = df.grapheme_root.values
    self.vowel_diacritic =df.vowel_diacritic.values 
    self.consonant_diacritic = df.consonant_diacritic.values
    if len(folds)==1 : 
      self.aug = albumentations.Compose([albumentations.Resize(img_height,img_width)
                                        ,albumentations.Normalize(mean,std,always_apply=True)
                                         ]) 
    else :     
      self.aug = albumentations.Compose([albumentations.Resize(img_height,img_width)
                                        ,albumentations.ShiftScaleRotate(shift_limit=0.0625,scale_limit=0.1,rotate_limit=5,p=0.9)
                                        ,albumentations.Normalize(mean,std,always_apply=True)
                                         ])
  def __len__(self) : 
    return (len(self.image_ids)) 
  def __getitem__(self,item) : 
    image = joblib.load(f"/content/Kaggle/images/{self.image_ids[item]}.pkl")
    image = image.reshape(137,236).astype(float) 
    image = Image.fromarray(image).convert('RGB')
    image = self.aug(image= np.array(image))['image']
    image = np.transpose(image,(2,0,1)).astype(float)
    return { 'image' : torch.tensor(image,dtype = torch.float),
             'grapheme_root' : torch.tensor(self.grapheme_root[item],dtype = torch.long),
             'vowel_diacritic' : torch.tensor(self.vowel_diacritic[item],dtype = torch.long),
             'consonant_diacritic' : torch.tensor(self.consonant_diacritic[item],dtype = torch.long)
              }


In [0]:
dataset = BengaliDatasetTrain(folds=[0,1])

In [0]:
len(dataset)

80336

In [0]:
import matplotlib.pyplot as plt 
idx = 5 
img = dataset[idx]['image']
print(dataset[idx]['grapheme_root'])
print(dataset[idx]['vowel_diacritic'])
print(dataset[idx]['consonant_diacritic'])
npimg = img.numpy() 
plt.imshow(np.transpose(npimg,(1,2,0)))


FileNotFoundError: ignored

# Modeling

In [0]:
!pip install pretrainedmodels

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60962 sha256=7fbd80ee91bbd2924c1a19532cef14d2b3c52add5f80d7a93cd4e8447c7a0800
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels


In [0]:
import pretrainedmodels 
import torch.nn as nn 
from torch.nn import functional as F
class Inceptionv3(nn.Module) : 
  def __init__(self,pretrained) : 
    super(Inceptionv3,self).__init__() 
    if pretrained is True : 
      self.model = pretrainedmodels.__dict__['inceptionv3'](pretrained = 'imagenet') 
    else : 
      self.model = pretrainedmodels.__dict__['inceptionv3'](pretrained = None) 
    self.l0 = nn.Linear(512,168) 
    self.l1 = nn.Linear(512,11)
    self.l2 = nn.Linear(512,7) 
  def forward(self,x) : 
    bs,_,_,_ = x.shape
    x = self.model.features(x) 
    x = F.adaptive_avg_pool2d(x,1).reshape(bs,-1)
    l0 = self.l0(x)
    l1 = self.l1(x)
    l2 = self.l2(x)
    return l0,l1,l2

In [0]:

class ResNet101(nn.Module) : 
  def __init__(self,pretrained) : 
    super(ResNet101,self).__init__() 
    if pretrained is True : 
      self.model = pretrainedmodels.__dict__['resnet101'](pretrained = 'imagenet') 
    else : 
      self.model = pretrainedmodels.__dict__['resnet101'](pretrained = None) 
    self.l0 = nn.Linear(2048,168) 
    self.l1 = nn.Linear(2048,11)
    self.l2 = nn.Linear(2048,7) 
  def forward(self,x) : 
    bs,_,_,_ = x.shape
    x = self.model.features(x) 
    x = F.adaptive_avg_pool2d(x,1).reshape(bs,-1)
    l0 = self.l0(x)
    l1 = self.l1(x)
    l2 = self.l2(x)
    return l0,l1,l2

NameError: ignored

In [0]:
class ResNet34(nn.Module) : 
  def __init__(self,pretrained) : 
    super(ResNet34,self).__init__() 
    if pretrained is True : 
      self.model = pretrainedmodels.__dict__['resnet34'](pretrained = 'imagenet') 
    else : 
      self.model = pretrainedmodels.__dict__['resnet34'](pretrained = None) 
    self.l0 = nn.Linear(512,168) 
    self.l1 = nn.Linear(512,11)
    self.l2 = nn.Linear(512,7) 
  def forward(self,x) : 
    bs,_,_,_ = x.shape
    x = self.model.features(x) 
    x = F.adaptive_avg_pool2d(x,1).reshape(bs,-1)
    l0 = self.l0(x)
    l1 = self.l1(x)
    l2 = self.l2(x)
    return l0,l1,l2

In [0]:
def loss_fn(outputs,targets) : 
  o1,o2,o3=outputs
  t1,t2,t3 =targets 
  l1 =nn.CrossEntropyLoss()(o1,t1)
  l2 =nn.CrossEntropyLoss()(o2,t2)
  l3 =nn.CrossEntropyLoss()(o3,t3)
  return (l1+l2+l3) / 3 

In [0]:
from tqdm import tqdm
def train (dataset,data_loader,model,optimizer) : 
  model.train() 
  for bi,d in tqdm(enumerate(data_loader),total = int(len(dataset)/data_loader.batch_size)):
    
    img = d['image']
    grapheme_root=d['grapheme_root']
    vowel_diacritic=d['vowel_diacritic']
    consonant_diacritic=d['consonant_diacritic']
    
    
    img =img.to(DEVICE,dtype=torch.float)
    grapheme_root=grapheme_root.to(DEVICE,dtype=torch.long )
    vowel_diacritic=vowel_diacritic.to(DEVICE,dtype=torch.long )
    consonant_diacritic=consonant_diacritic.to(DEVICE,dtype=torch.long )

    optimizer.zero_grad()
    outputs =model(img) 

    target = (grapheme_root,vowel_diacritic,consonant_diacritic)
    loss =loss_fn(outputs,target) 
    loss.backward()
    optimizer.step()


In [0]:
def evaluate (dataset,data_loader,model) : 
  with torch.no_grad():
    model.to(DEVICE)
    model.eval() 
    final_loss = 0
    counter=0
    for bi,d in tqdm(enumerate(data_loader),total = int(len(dataset)/data_loader.batch_size)):
      counter=counter+1 
      img = d['image']
      grapheme_root=d['grapheme_root']
      vowel_diacritic=d['vowel_diacritic']
      consonant_diacritic=d['consonant_diacritic']
    
    
      img =img.to(DEVICE,dtype=torch.float)
      grapheme_root=grapheme_root.to(DEVICE,dtype=torch.long )
      vowel_diacritic=vowel_diacritic.to(DEVICE,dtype=torch.long )
      consonant_diacritic=consonant_diacritic.to(DEVICE,dtype=torch.long )

      outputs =model(img) 
      target = (grapheme_root,vowel_diacritic,consonant_diacritic)
      loss =loss_fn(outputs,target) 
      final_loss+= loss 
    return final_loss/counter

In [0]:
DEVICE ='cuda'
CUDA_VISIBLE_DEVICES =0
EPOCHS = 40
TRAIN_BATCH_SIZE = 96
VAL_BATCH_SIZE = 32 
TRAINING_FOLDS  = [(0,1,2,3),(0,4,2,3),(0,1,4,3),(0,1,2,4)]
VALIDATION_FOLDS =((4,),(1,),(2,),(3,))

In [0]:
model = ResNet101(pretrained=False)
model.load_state_dict(torch.load("/content/drive/My Drive/Kaggle/model2_2.bin"))
model.eval()
model = model.to(DEVICE)

In [0]:
for i in  range(len(TRAINING_FOLDS)) :
  train_dataset = BengaliDatasetTrain(folds=TRAINING_FOLDS[i])
  train_loder = torch.utils.data.DataLoader(
    dataset = train_dataset , 
    batch_size = TRAIN_BATCH_SIZE,
    shuffle = True , 
    num_workers = 4
)
  valid_dataset = BengaliDatasetTrain(folds=VALIDATION_FOLDS[i])
  valid_loder = torch.utils.data.DataLoader(
    dataset = valid_dataset , 
    batch_size = VAL_BATCH_SIZE,
    shuffle = False , 
    num_workers = 4
)
  optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='max',patience= 5,factor=0.3,verbose=True )
  if torch.cuda.device_count()> 1 : 
    model = nn.DataParallel(model)
  for epoch in range(EPOCHS) : 
    print(epoch)
    train(train_dataset,train_loder,model,optimizer)
    val_score =evaluate(valid_dataset,valid_loder,model)
    print('val_score= ',val_score)
    scheduler.step(val_score) 
    torch.save(model.state_dict(),f"/content/drive/My Drive/Kaggle/model_{i}.bin")  

0



100%|██████████| 1673/1673 [1:11:06<00:00,  2.55s/it]
1674it [1:11:08,  2.32s/it]                          

100%|██████████| 1255/1255 [06:00<00:00,  3.46it/s]
1256it [06:00,  2.07it/s]                          


val_score=  tensor(0.0764, device='cuda:0')
1



 28%|██▊       | 467/1673 [19:52<51:13,  2.55s/it]

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=ebffaf7c5b003967191db0af0f5b486cc70dafbb064a30165e662488b55957df
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 7.1 GB  | Proc size: 6.8 GB
GPU RAM Free: 4281MB | Used: 7160MB | Util  63% | Total 11441MB


In [0]:
!kill -9 -1